# Estudo de Caso: Alocação de Ativos em Carteira de Investimentos

Monte uma carteira de ações para o mês de Setembro/2020 baseado na Moderna Teoria de Portfolio (Markowitz, 1952), a qual estuda a melhor combinação possível dos ativos analisados e sugere uma alocação de ativos dentro de uma carteira que maximize o retorno. 

Considere as cotações históricas das ações que fazem parte da [Carteira Teórica do Ibovespa em 01/09/2020](http://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/indice-ibovespa-ibovespa-composicao-da-carteira.htm) disponíveis em [ibovespa_desc.csv](ibovespa_desc.csv) e no diretório ibovespa.

Caso queira consultar o retorno histórico diário, semanal ou mensal acesse o [link](https://www.investing.com/equities/ambev-pn-historical-data).

In [1]:
#include <iostream>
#include <iomanip>

#include "csv.h"
#include "setup.h"

#include "ortools/linear_solver/linear_solver.h"
#include "ortools/linear_solver/linear_solver.pb.h"

using namespace operations_research;

## Símbolos de uma carteira

Use a função getTickersByStartDate para retornar a lista de símbolos dos ativos existentes a partir de um determinado mês e ano.

In [2]:
int getTickersByStartDate(int year, int month, std::vector<std::string> & selectedTickers)
{
    int months(0);
    std::string initial_date(std::to_string(year) + "-" + (month<10 ? "0" + std::to_string(month) : std::to_string(month)) );
    std::ifstream f1("ibovespa_desc.csv");
    auto tickers = csv_read_string(f1);
    transpose<std::string>(tickers);
    f1.close();

    for(int ticker = 0; ticker < tickers[0].size(); ticker++){
        auto filename = std::string("ibovespa/") + tickers[0][ticker] + "M.csv";
        std::ifstream f2(filename);
        auto returns = csv_read_string(f2);
        transpose<std::string>(returns);
        f2.close();
        
        int date_offset = -1;
        for(int month=0; month<returns[0].size(); month++){
            if (returns[0][month] == initial_date){
                date_offset = month;
                break;
            } 
            else if (returns[0][month] > initial_date){
                date_offset = -1;
                break;
            }
        }
        
        if (date_offset >= 0) {
            months = returns[0].size()-date_offset;
            selectedTickers.push_back(tickers[0][ticker]);
        }

    }
    return months;
}

In [3]:
std::vector<std::string> simbolos;
int meses = getTickersByStartDate(2010, 1, simbolos);

In [4]:
simbolos

{ "ABEV3", "BBAS3", "BBDC3", "BBDC4", "BEEF3", "BRAP4", "BRFS3", "BRKM5", "BRML3", "BTOW3", "CCRO3", "CIEL3", "CMIG4", "CPFE3", "CSAN3", "CSNA3", "CYRE3", "EGIE3", "ELET3", "EMBR3", "ENBR3", "ENGI11", "EQTL3", "FLRY3", "GGBR4", "GOAU4", "GOLL4", "HGTX3", "HYPE3", "IGTA3", "ITSA4", "ITUB4", "JBSS3", "LAME4", "LREN3", "MRFG3", "MRVE3", "MULT3", "PETR3", "PETR4", "RADL3", "RENT3", "SANB11", "SBSP3", "SULA11", "TIMP3", "TOTS3", "USIM5", "VALE3", "VIVT4", "VVAR3", "WEGE3" }

## Retorno de uma carteira

Calcule o retorno médio mensal em percentual de cada ação baseado nos dados históricos fornecidos. Fica a critério de cada grupo escolher o intervalo mais apropriado de datas.

    O algoritmo abaixo seleciona somente as ações de empresas listadas na Bovespa a partir de uma data configurada (initial_date).

In [5]:
void getReturnsByTicker(const std::vector<std::string> & tickers, std::vector<std::vector<double>> & selectedReturns)
{
    for(int ticker = 0; ticker < tickers.size(); ticker++){
        auto filename = std::string("ibovespa/") + tickers[ticker] + "M.csv";
        std::ifstream f2(filename);
        auto returns = csv_read_string(f2);
        transpose<std::string>(returns);
        f2.close();
        
        int date_offset = returns[1].size() - selectedReturns[ticker].size();
        
        if (date_offset >= 0) {
            for(int month=date_offset; month<returns[1].size(); month++) {
                selectedReturns[ticker][month-date_offset] = std::stof(returns[1][month]);
            }
        }

    }

}

In [6]:
std::vector<std::vector<double>> retornos(simbolos.size(), std::vector<double>(meses));
getReturnsByTicker(simbolos, retornos);

In [8]:
retornos[0] //retornos mensais em percentual da ABEV3

{ -0.019608514, -0.0057999501, -0.010329015, 0.065854736, 0.0012233452, 0.059063159, 0.046347015, 0.023096217, 0.16563351, 0.11693622, 0.029861657, 0.12658258, -0.12511468, -0.013695969, 0.11612678, 0.087113075, -0.024711153, 0.045719758, -0.074742764, 0.20370425, 0.010768813, 0.081211194, 0.069887564, 0.10080157, -0.047615353, 0.11769694, 0.15854691, 0.071139045, -0.071503103, 0.021451183, 0.018051369, -0.0082436679, -0.00015643575, 0.11615376, 0.13762739, 0.13856943, 0.075628862, 0.0073577561, -0.061519045, -0.0014649433, 0.0030563935, 0.010116739, 0.037521601, -0.028953817, 0.054700974, -0.014520727, 0.052125886, -0.013668721, -0.065516181, 0.049097169, 0.0041473866, -0.033700261, -0.028186282, -0.0063050650, 0.0041397056, 0.044671539, -0.020159001, 0.037069116, 0.021951372, -0.028042989, 0.10106125, 0.035047892, 0.014270224, 0.025432952, -0.022163907, 0.030364348, 0.025276536, -0.019517304, 0.025127962, -0.016992785, -0.023572356, -0.034334719, 0.045377929, -0.048030280, 0.06575956

# Retorno Médio e Risco de uma carteira

Calcule a matriz de covariância considerando os retornos selecionados no passo anterior. 

Suponha que sejam consideradas $n$ ações para inclusão nessa carteira e façamos com que as variáveis de decisão $x_i$ ($i = 1, 2, ..., n$) representem o percentual da carteira que será alocado no ativo $i$. 

Estipulamos que $\mu_i$ e $\sigma_{ii}$ sejam, respectivamente, a média e a variância, (estimadas) do retorno sobre cada cota da ação i, em que $\sigma_{ii}$ mede o risco dessa ação.

Para $i=1,2,...,n$ ($i \ne j$),façamos com que $\sigma_{ij}$ represente a co-variância do retorno sobre cada cota da ação i e j. 

Como seria difícil estimar todos os valores $\sigma_{ij}$, a metodologia usual é partir de certas hipóteses sobre o comportamento do mercado que nos permitam calcular $\sigma_{ij}$ diretamente de $\sigma_{ii}$ e $\sigma_{jj}$.

A seguir, o valor esperado $R(x)$ e a variância $V(x)$ do retorno total de toda a carteira são:

$R(x) = \sum_{j=1}^n \mu_{j}x_j$

e

$V(x) = \sum_{i=1}^n \sum_{j=1}^n \sigma_{ij}x_i x_j$

em que $V(x)$ mede o risco associado à carteira com base na matriz de covariância $\Sigma$, $N\times N$:

$$ \Sigma = \left[\begin{matrix}
VAR(R_1) & COV(R_1, R_2) & \cdots & COV(R_1, R_N) \\
COV(R_2, R_0) & VAR(R_2) & \cdots & COV(R_2, R_N) \\
\vdots & \vdots & \ddots & \vdots \\
COV(R_N, R_1) & COV(R_N, R_2) & \cdots & VAR(R_N)
\end{matrix}\right] $$


## Otimização de Carteira de Investimentos

O objetivo é analisar as cotações históricas das ações do Ibovespa segundo a Moderna Teoria de Portfolio para gerar uma distribuição de alocação ótima (em percentual) composta de 5 a 10 ativos para ser utilizada na carteira mensal de Setembro de 2020. 

**Importante:** Os grupos cujas carteiras otimizadas alcançarem os maiores retornos (positivos) em Setembro/2020 receberão pontuação extra proporcional ao rendimento alcançado.


## Escolha uma das duas modelagens abaixo e justifique:

Minimizar: $x^T\sum x$

Minimizar: $\sum_{i=1}^n \sum_{j=1}^n \sigma_{ij}x_i x_j$


Sujeito a: 

$
\begin{align}
\mu^Tx \ge r_{min} \\
1^Tx \le 1  \\
x \ge 0
\end{align}
$


Maximizar: $\mu^Tx - \rho x^T\sum x$

Maximizar: $\sum_{i=1}^n \mu_i x_i - \rho \sum_{i=1}^n \sum_{j=1}^n \sigma_{ij}x_i x_j$


Sujeito a: 

$
\begin{align}
1^Tx \le 1  \\
x \ge 0
\end{align}
$

## A otimização deverá retornar o vetor $x$ contendo apenas entre 5 e 10 ativos selecionados automaticamente pelo solver.